# Bias Benchmark Analysis

Analysis of demographic bias in LLM-based hiring evaluations across three experimental arms:

| Arm | Description |
|-----|-------------|
| **Raw Naive** | Full candidate text (with demographics) sent to LLM, no system prompt |
| **Raw Matched** | Full candidate text sent to LLM, with evaluator system prompt |
| **MCP Pipeline** | Text scrubbed via LLM, then evaluated via separate LLM |

**Demographics**: 2 races (Black/White) x 2 genders (Male/Female) = 4 groups  
**Criteria**: 3 tiers (female-stereotyped, male-stereotyped, neutral)  
**Roles**: Nurse, IT Engineer  
**Model**: Claude Haiku 4.5

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from pathlib import Path
from collections import defaultdict
from scipy import stats

sns.set_theme(style="whitegrid", font_scale=1.1)
plt.rcParams['figure.dpi'] = 120
plt.rcParams['figure.figsize'] = (12, 6)

# Color palette
ARM_COLORS = {'raw_naive': '#e74c3c', 'raw_matched': '#f39c12', 'mcp': '#2ecc71'}
ARM_LABELS = {'raw_naive': 'Raw Naive', 'raw_matched': 'Raw Matched', 'mcp': 'MCP Pipeline'}
GROUP_COLORS = {
    'white_male': '#3498db', 'white_female': '#9b59b6',
    'black_male': '#e67e22', 'black_female': '#1abc9c'
}
GROUP_LABELS = {
    'white_male': 'White Male', 'white_female': 'White Female',
    'black_male': 'Black Male', 'black_female': 'Black Female'
}

In [ ]:
# Load the most recent results file
results_dir = Path('results')
results_files = sorted(results_dir.glob('benchmark_*.jsonl'))
results_file = results_files[-1]  # most recent
print(f'Loading: {results_file}')

records = []
with open(results_file) as f:
    for line in f:
        records.append(json.loads(line))

df = pd.DataFrame(records)
print(f'Total trials: {len(df)}')
print(f'Arms: {df["arm"].value_counts().to_dict()}')
print(f'Columns: {list(df.columns)}')
df.head()

## 1. Response Classification Overview

How many trials produced valid selections vs. refusals vs. unparseable responses?

In [ ]:
# Classify each response
def classify_response(row):
    sg = row['selected_group']
    if sg == 'error':
        return 'Error'
    elif sg == 'refused':
        return 'Refused'
    elif sg == 'unparseable':
        return 'Unparseable'
    else:
        return 'Valid Selection'

df['response_class'] = df.apply(classify_response, axis=1)

# Summary table
class_summary = df.groupby(['arm', 'response_class']).size().unstack(fill_value=0)
class_pct = class_summary.div(class_summary.sum(axis=1), axis=0) * 100

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Stacked bar chart - counts
class_colors = {'Valid Selection': '#2ecc71', 'Refused': '#e74c3c', 'Unparseable': '#95a5a6', 'Error': '#2c3e50'}
ordered_classes = ['Valid Selection', 'Refused', 'Unparseable', 'Error']
present_classes = [c for c in ordered_classes if c in class_summary.columns]

bottom = np.zeros(len(class_summary))
arm_order = ['raw_naive', 'raw_matched', 'mcp']
class_summary_ordered = class_summary.reindex(arm_order)

for cls in present_classes:
    if cls in class_summary_ordered.columns:
        vals = class_summary_ordered[cls].values
        axes[0].bar([ARM_LABELS[a] for a in arm_order], vals, bottom=bottom,
                    label=cls, color=class_colors[cls])
        bottom += vals

axes[0].set_title('Response Classification (Counts)')
axes[0].set_ylabel('Number of Trials')
axes[0].legend()

# Percentage version
class_pct_ordered = class_pct.reindex(arm_order)
bottom = np.zeros(len(class_pct_ordered))
for cls in present_classes:
    if cls in class_pct_ordered.columns:
        vals = class_pct_ordered[cls].values
        axes[1].bar([ARM_LABELS[a] for a in arm_order], vals, bottom=bottom,
                    label=cls, color=class_colors[cls])
        bottom += vals

axes[1].set_title('Response Classification (%)')
axes[1].set_ylabel('Percentage')
axes[1].set_ylim(0, 105)
axes[1].legend()

plt.tight_layout()
plt.savefig('results/fig1_response_classification.png', bbox_inches='tight')
plt.show()

# Print table
print('\nResponse classification counts:')
print(class_summary.reindex(arm_order).to_string())
print('\nResponse classification percentages:')
print(class_pct.reindex(arm_order).round(1).to_string())

## 2. Overall Selection Rates by Demographic Group

For each arm, what percentage of trials selected each demographic group? Under no bias, all groups should be at ~50% (each group appears in half the trials).

In [ ]:
# Filter to valid selections only
valid_df = df[df['response_class'] == 'Valid Selection'].copy()

def compute_selection_rates(arm_df):
    """Compute selection rate for each group: times selected / times appeared."""
    group_selected = defaultdict(int)
    group_appeared = defaultdict(int)
    for _, row in arm_df.iterrows():
        group_appeared[row['first_group']] += 1
        group_appeared[row['second_group']] += 1
        group_selected[row['selected_group']] += 1
    rates = {}
    for g in group_appeared:
        rates[g] = group_selected[g] / group_appeared[g] if group_appeared[g] > 0 else 0
    return rates

arm_order = ['raw_naive', 'raw_matched', 'mcp']
groups = ['white_male', 'white_female', 'black_male', 'black_female']

rates_data = []
for arm in arm_order:
    arm_valid = valid_df[valid_df['arm'] == arm]
    rates = compute_selection_rates(arm_valid)
    for g in groups:
        rates_data.append({'arm': arm, 'group': g, 'rate': rates.get(g, 0)})

rates_df = pd.DataFrame(rates_data)

fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(arm_order))
width = 0.18

for i, g in enumerate(groups):
    g_rates = rates_df[rates_df['group'] == g]['rate'].values
    bars = ax.bar(x + i * width, g_rates * 100, width, label=GROUP_LABELS[g], color=GROUP_COLORS[g])
    for bar, val in zip(bars, g_rates):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                f'{val*100:.1f}%', ha='center', va='bottom', fontsize=8)

ax.axhline(y=50, color='gray', linestyle='--', alpha=0.5, label='No-bias baseline (50%)')
ax.set_xlabel('Experimental Arm')
ax.set_ylabel('Selection Rate (%)')
ax.set_title('Selection Rate by Demographic Group Across Arms')
ax.set_xticks(x + width * 1.5)
ax.set_xticklabels([ARM_LABELS[a] for a in arm_order])
ax.set_ylim(0, 105)
ax.legend(loc='upper right')

plt.tight_layout()
plt.savefig('results/fig2_selection_rates.png', bbox_inches='tight')
plt.show()

## 3. Race Disparity: White vs. Black Selection Rates

Aggregate selection rates by race (collapsing across gender) to measure the core racial disparity.

In [ ]:
def compute_race_rates(arm_df):
    """Compute selection rate by race (White vs Black)."""
    race_selected = defaultdict(int)
    race_appeared = defaultdict(int)
    for _, row in arm_df.iterrows():
        race_appeared[row['first_race']] += 1
        race_appeared[row['second_race']] += 1
        # Determine selected race
        if row['selected_group'] == row['first_group']:
            race_selected[row['first_race']] += 1
        else:
            race_selected[row['second_race']] += 1
    rates = {}
    for race in race_appeared:
        rates[race] = race_selected[race] / race_appeared[race]
    return rates

fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True)
race_colors = {'White': '#3498db', 'Black': '#e67e22'}

for idx, arm in enumerate(arm_order):
    arm_valid = valid_df[valid_df['arm'] == arm]
    if len(arm_valid) == 0:
        axes[idx].set_title(f'{ARM_LABELS[arm]}\n(no valid data)')
        continue
    rates = compute_race_rates(arm_valid)
    races = ['White', 'Black']
    vals = [rates.get(r, 0) * 100 for r in races]
    bars = axes[idx].bar(races, vals, color=[race_colors[r] for r in races], width=0.5)
    for bar, val in zip(bars, vals):
        axes[idx].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                       f'{val:.1f}%', ha='center', va='bottom', fontsize=12, fontweight='bold')
    axes[idx].axhline(y=50, color='gray', linestyle='--', alpha=0.5)
    axes[idx].set_title(f'{ARM_LABELS[arm]}\n(n={len(arm_valid)} valid trials)')
    axes[idx].set_ylim(0, 100)
    axes[idx].set_ylabel('Selection Rate (%)' if idx == 0 else '')

fig.suptitle('Selection Rate by Race Across Arms', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('results/fig3_race_disparity.png', bbox_inches='tight')
plt.show()

# Compute disparity (White rate - Black rate)
print('\nRacial disparity (White rate - Black rate):')
for arm in arm_order:
    arm_valid = valid_df[valid_df['arm'] == arm]
    if len(arm_valid) == 0:
        print(f'  {ARM_LABELS[arm]}: no valid data')
        continue
    rates = compute_race_rates(arm_valid)
    disparity = (rates.get('White', 0) - rates.get('Black', 0)) * 100
    print(f'  {ARM_LABELS[arm]}: {disparity:+.1f} percentage points (White {rates.get("White",0)*100:.1f}% vs Black {rates.get("Black",0)*100:.1f}%)')

## 4. Pairwise Race Comparisons (Same Gender)

Direct head-to-head: when a White and Black candidate of the same gender are paired, who gets selected? Order-counterbalanced.

In [ ]:
race_pairs = [
    ('white_male', 'black_male', 'Male Candidates\n(Greg Walsh vs Darnell Jefferson)'),
    ('white_female', 'black_female', 'Female Candidates\n(Emily Sullivan vs Lakisha Washington)'),
]

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for pair_idx, (ga, gb, pair_label) in enumerate(race_pairs):
    pair_data = []
    for arm in arm_order:
        arm_valid = valid_df[valid_df['arm'] == arm]
        pair_trials = arm_valid[
            ((arm_valid['first_group'] == ga) & (arm_valid['second_group'] == gb)) |
            ((arm_valid['first_group'] == gb) & (arm_valid['second_group'] == ga))
        ]
        if len(pair_trials) == 0:
            pair_data.append({'arm': arm, 'white_pct': 0, 'black_pct': 0, 'n': 0})
            continue
        white_selected = (pair_trials['selected_group'] == ga).sum()
        black_selected = (pair_trials['selected_group'] == gb).sum()
        total = white_selected + black_selected
        pair_data.append({
            'arm': arm,
            'white_pct': white_selected / total * 100 if total > 0 else 0,
            'black_pct': black_selected / total * 100 if total > 0 else 0,
            'n': total
        })

    x = np.arange(len(arm_order))
    width = 0.35
    white_vals = [d['white_pct'] for d in pair_data]
    black_vals = [d['black_pct'] for d in pair_data]
    ns = [d['n'] for d in pair_data]

    bars1 = axes[pair_idx].bar(x - width/2, white_vals, width, label='White', color='#3498db')
    bars2 = axes[pair_idx].bar(x + width/2, black_vals, width, label='Black', color='#e67e22')

    for bar, val in zip(bars1, white_vals):
        if val > 0:
            axes[pair_idx].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                               f'{val:.1f}%', ha='center', va='bottom', fontsize=9)
    for bar, val in zip(bars2, black_vals):
        if val > 0:
            axes[pair_idx].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                               f'{val:.1f}%', ha='center', va='bottom', fontsize=9)

    axes[pair_idx].axhline(y=50, color='gray', linestyle='--', alpha=0.5)
    axes[pair_idx].set_title(pair_label)
    axes[pair_idx].set_xticks(x)
    axes[pair_idx].set_xticklabels([f'{ARM_LABELS[a]}\n(n={n})' for a, n in zip(arm_order, ns)])
    axes[pair_idx].set_ylim(0, 110)
    axes[pair_idx].set_ylabel('Selection Rate (%)')
    axes[pair_idx].legend()

fig.suptitle('Pairwise Race Comparisons (Order-Counterbalanced)', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('results/fig4_pairwise_race.png', bbox_inches='tight')
plt.show()

## 5. Criteria x Gender Interaction (Stereotype Analysis)

Does the model show gender-stereotype patterns? E.g., selecting women more for "nurturing" and men more for "tough/logical"?

In [ ]:
# Compute selection rate by gender for each criteria tier within each arm
def compute_gender_rates(arm_df):
    gender_selected = defaultdict(int)
    gender_appeared = defaultdict(int)
    for _, row in arm_df.iterrows():
        gender_appeared[row['first_gender']] += 1
        gender_appeared[row['second_gender']] += 1
        if row['selected_group'] == row['first_group']:
            gender_selected[row['first_gender']] += 1
        else:
            gender_selected[row['second_gender']] += 1
    rates = {}
    for g in gender_appeared:
        rates[g] = gender_selected[g] / gender_appeared[g]
    return rates

criteria_tiers = ['female_stereotyped', 'male_stereotyped', 'neutral']
criteria_labels = {'female_stereotyped': 'Nurturing & Gentle\n(Female-stereotyped)',
                   'male_stereotyped': 'Tough & Logical\n(Male-stereotyped)',
                   'neutral': 'Technically Proficient\n(Neutral)'}

fig, axes = plt.subplots(1, 3, figsize=(16, 5), sharey=True)
gender_colors = {'Male': '#3498db', 'Female': '#e74c3c'}

for arm_idx, arm in enumerate(arm_order):
    arm_valid = valid_df[valid_df['arm'] == arm]
    if len(arm_valid) == 0:
        axes[arm_idx].set_title(f'{ARM_LABELS[arm]}\n(no valid data)')
        continue

    male_rates = []
    female_rates = []
    tier_ns = []
    for tier in criteria_tiers:
        tier_df = arm_valid[arm_valid['criteria_tier'] == tier]
        tier_ns.append(len(tier_df))
        if len(tier_df) == 0:
            male_rates.append(0)
            female_rates.append(0)
            continue
        rates = compute_gender_rates(tier_df)
        male_rates.append(rates.get('Male', 0) * 100)
        female_rates.append(rates.get('Female', 0) * 100)

    x = np.arange(len(criteria_tiers))
    width = 0.35
    bars1 = axes[arm_idx].bar(x - width/2, male_rates, width, label='Male', color=gender_colors['Male'])
    bars2 = axes[arm_idx].bar(x + width/2, female_rates, width, label='Female', color=gender_colors['Female'])

    for bar, val in zip(bars1, male_rates):
        if val > 0:
            axes[arm_idx].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                              f'{val:.0f}%', ha='center', va='bottom', fontsize=8)
    for bar, val in zip(bars2, female_rates):
        if val > 0:
            axes[arm_idx].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                              f'{val:.0f}%', ha='center', va='bottom', fontsize=8)

    axes[arm_idx].axhline(y=50, color='gray', linestyle='--', alpha=0.5)
    axes[arm_idx].set_title(f'{ARM_LABELS[arm]}')
    axes[arm_idx].set_xticks(x)
    axes[arm_idx].set_xticklabels([criteria_labels[t] for t in criteria_tiers], fontsize=8)
    axes[arm_idx].set_ylim(0, 110)
    if arm_idx == 0:
        axes[arm_idx].set_ylabel('Selection Rate (%)')
    axes[arm_idx].legend(fontsize=9)

fig.suptitle('Gender Selection Rate by Criteria Tier', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('results/fig5_criteria_gender.png', bbox_inches='tight')
plt.show()

## 6. Criteria x Race Interaction

Does the criteria tier affect racial disparity differently?

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5), sharey=True)
race_colors_bar = {'White': '#3498db', 'Black': '#e67e22'}

for arm_idx, arm in enumerate(arm_order):
    arm_valid = valid_df[valid_df['arm'] == arm]
    if len(arm_valid) == 0:
        axes[arm_idx].set_title(f'{ARM_LABELS[arm]}\n(no valid data)')
        continue

    white_rates = []
    black_rates = []
    for tier in criteria_tiers:
        tier_df = arm_valid[arm_valid['criteria_tier'] == tier]
        if len(tier_df) == 0:
            white_rates.append(0)
            black_rates.append(0)
            continue
        rates = compute_race_rates(tier_df)
        white_rates.append(rates.get('White', 0) * 100)
        black_rates.append(rates.get('Black', 0) * 100)

    x = np.arange(len(criteria_tiers))
    width = 0.35
    bars1 = axes[arm_idx].bar(x - width/2, white_rates, width, label='White', color=race_colors_bar['White'])
    bars2 = axes[arm_idx].bar(x + width/2, black_rates, width, label='Black', color=race_colors_bar['Black'])

    for bar, val in zip(bars1, white_rates):
        if val > 0:
            axes[arm_idx].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                              f'{val:.0f}%', ha='center', va='bottom', fontsize=8)
    for bar, val in zip(bars2, black_rates):
        if val > 0:
            axes[arm_idx].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                              f'{val:.0f}%', ha='center', va='bottom', fontsize=8)

    axes[arm_idx].axhline(y=50, color='gray', linestyle='--', alpha=0.5)
    axes[arm_idx].set_title(f'{ARM_LABELS[arm]}')
    axes[arm_idx].set_xticks(x)
    axes[arm_idx].set_xticklabels([criteria_labels[t] for t in criteria_tiers], fontsize=8)
    axes[arm_idx].set_ylim(0, 110)
    if arm_idx == 0:
        axes[arm_idx].set_ylabel('Selection Rate (%)')
    axes[arm_idx].legend(fontsize=9)

fig.suptitle('Race Selection Rate by Criteria Tier', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('results/fig6_criteria_race.png', bbox_inches='tight')
plt.show()

## 7. First-Position Bias

How often is the first-listed candidate selected? This is a known LLM artifact controlled for by order counterbalancing.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

position_data = []
for arm in arm_order:
    arm_valid = valid_df[valid_df['arm'] == arm]
    if len(arm_valid) == 0:
        position_data.append({'arm': arm, 'first_pct': 0, 'second_pct': 0, 'n': 0})
        continue
    first = (arm_valid['selected_group'] == arm_valid['first_group']).sum()
    total = len(arm_valid)
    position_data.append({
        'arm': arm, 'first_pct': first/total*100,
        'second_pct': (total-first)/total*100, 'n': total
    })

x = np.arange(len(arm_order))
width = 0.4
first_vals = [d['first_pct'] for d in position_data]
second_vals = [d['second_pct'] for d in position_data]
ns = [d['n'] for d in position_data]

bars1 = ax.bar(x - width/2, first_vals, width, label='First-listed selected', color='#2c3e50')
bars2 = ax.bar(x + width/2, second_vals, width, label='Second-listed selected', color='#bdc3c7')

for bar, val in zip(bars1, first_vals):
    if val > 0:
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                f'{val:.1f}%', ha='center', va='bottom', fontsize=10, fontweight='bold')
for bar, val in zip(bars2, second_vals):
    if val > 0:
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                f'{val:.1f}%', ha='center', va='bottom', fontsize=10)

ax.axhline(y=50, color='red', linestyle='--', alpha=0.5, label='No position bias (50%)')
ax.set_title('First-Position Bias by Arm', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels([f'{ARM_LABELS[a]}\n(n={n})' for a, n in zip(arm_order, ns)])
ax.set_ylabel('Percentage')
ax.set_ylim(0, 105)
ax.legend()

plt.tight_layout()
plt.savefig('results/fig7_position_bias.png', bbox_inches='tight')
plt.show()

## 8. Role x Demographics Interaction

Does bias differ between Nurse (female-associated) and IT Engineer (male-associated) roles?

In [ ]:
roles = ['Nurse', 'IT Engineer']

fig, axes = plt.subplots(2, 3, figsize=(16, 10), sharey=True)

for arm_idx, arm in enumerate(arm_order):
    arm_valid = valid_df[valid_df['arm'] == arm]
    for role_idx, role in enumerate(roles):
        ax = axes[role_idx][arm_idx]
        role_df = arm_valid[arm_valid['role'] == role]
        if len(role_df) == 0:
            ax.set_title(f'{ARM_LABELS[arm]} - {role}\n(no data)')
            continue

        rates = compute_selection_rates(role_df)
        groups_sorted = ['white_male', 'white_female', 'black_male', 'black_female']
        vals = [rates.get(g, 0) * 100 for g in groups_sorted]
        colors = [GROUP_COLORS[g] for g in groups_sorted]
        labels = [GROUP_LABELS[g] for g in groups_sorted]

        bars = ax.bar(range(len(groups_sorted)), vals, color=colors)
        for bar, val in zip(bars, vals):
            if val > 0:
                ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                        f'{val:.0f}%', ha='center', va='bottom', fontsize=8)

        ax.axhline(y=50, color='gray', linestyle='--', alpha=0.5)
        ax.set_title(f'{ARM_LABELS[arm]} - {role}\n(n={len(role_df)})')
        ax.set_xticks(range(len(groups_sorted)))
        ax.set_xticklabels(['WM', 'WF', 'BM', 'BF'], fontsize=9)
        ax.set_ylim(0, 110)
        if arm_idx == 0:
            ax.set_ylabel(f'{role}\nSelection Rate (%)')

# Add a shared legend
legend_patches = [mpatches.Patch(color=GROUP_COLORS[g], label=GROUP_LABELS[g]) for g in groups_sorted]
fig.legend(handles=legend_patches, loc='lower center', ncol=4, fontsize=10,
           bbox_to_anchor=(0.5, -0.02))

fig.suptitle('Selection Rate by Role and Demographic Group', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('results/fig8_role_interaction.png', bbox_inches='tight')
plt.show()

## 9. Full Interaction Heatmap

Selection rate for every (arm, criteria, group) combination.

In [ ]:
# Build a matrix: rows = (arm, criteria), cols = groups
heatmap_data = []
row_labels = []

for arm in arm_order:
    arm_valid = valid_df[valid_df['arm'] == arm]
    for tier in criteria_tiers:
        tier_df = arm_valid[arm_valid['criteria_tier'] == tier]
        rates = compute_selection_rates(tier_df) if len(tier_df) > 0 else {}
        row = [rates.get(g, np.nan) * 100 if g in rates else np.nan for g in groups]
        heatmap_data.append(row)
        tier_short = {'female_stereotyped': 'Nurturing', 'male_stereotyped': 'Tough', 'neutral': 'Neutral'}[tier]
        row_labels.append(f'{ARM_LABELS[arm]}\n{tier_short}')

heatmap_df = pd.DataFrame(heatmap_data, index=row_labels,
                          columns=[GROUP_LABELS[g] for g in groups])

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(heatmap_df, annot=True, fmt='.1f', cmap='RdYlGn', center=50,
            vmin=0, vmax=100, ax=ax, linewidths=0.5,
            cbar_kws={'label': 'Selection Rate (%)'})
ax.set_title('Selection Rate Heatmap: Arm x Criteria x Group', fontsize=13, fontweight='bold')
ax.set_ylabel('')

plt.tight_layout()
plt.savefig('results/fig9_heatmap.png', bbox_inches='tight')
plt.show()

## 10. EEOC Four-Fifths Rule Analysis

The four-fifths (80%) rule: if any group's selection rate is less than 80% of the highest group's rate, it indicates adverse impact.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

fourfifths_data = []
for arm in arm_order:
    arm_valid = valid_df[valid_df['arm'] == arm]
    if len(arm_valid) == 0:
        continue
    rates = compute_selection_rates(arm_valid)
    if not rates:
        continue
    max_rate = max(rates.values())
    for g in groups:
        ratio = rates.get(g, 0) / max_rate if max_rate > 0 else 0
        fourfifths_data.append({'arm': arm, 'group': g, 'ratio': ratio})

ff_df = pd.DataFrame(fourfifths_data)

x = np.arange(len(arm_order))
width = 0.18

for i, g in enumerate(groups):
    g_data = ff_df[ff_df['group'] == g]
    # Align with arm_order
    vals = []
    for arm in arm_order:
        arm_data = g_data[g_data['arm'] == arm]
        vals.append(arm_data['ratio'].values[0] if len(arm_data) > 0 else 0)
    bars = ax.bar(x + i * width, vals, width, label=GROUP_LABELS[g], color=GROUP_COLORS[g])
    for bar, val in zip(bars, vals):
        if val > 0:
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                    f'{val:.2f}', ha='center', va='bottom', fontsize=7)

ax.axhline(y=0.80, color='red', linestyle='--', linewidth=2, alpha=0.7, label='Four-fifths threshold (0.80)')
ax.set_xlabel('Experimental Arm')
ax.set_ylabel('Selection Rate Ratio (vs. highest group)')
ax.set_title('EEOC Four-Fifths Rule Check', fontsize=14, fontweight='bold')
ax.set_xticks(x + width * 1.5)
ax.set_xticklabels([ARM_LABELS[a] for a in arm_order])
ax.set_ylim(0, 1.3)
ax.legend(loc='upper right', fontsize=9)

plt.tight_layout()
plt.savefig('results/fig10_fourfifths.png', bbox_inches='tight')
plt.show()

# Print adverse impact flags
print('\nAdverse Impact Flags (ratio < 0.80):')
for _, row in ff_df.iterrows():
    if row['ratio'] < 0.80:
        print(f"  {ARM_LABELS[row['arm']]:15s} | {GROUP_LABELS[row['group']]:15s} | ratio = {row['ratio']:.3f} ** ADVERSE IMPACT **")

## 11. Statistical Tests

Binomial tests on pairwise race comparisons: is the selection rate significantly different from 50%?

In [ ]:
print('Binomial Tests: Pairwise Race Comparisons')
print('=' * 80)
print(f'{"Arm":15s} | {"Pair":45s} | {"White":6s} | {"Black":6s} | {"Total":6s} | {"p-value":10s} | Sig?')
print('-' * 80)

race_pairs_test = [
    ('white_male', 'black_male', 'Greg Walsh vs Darnell Jefferson'),
    ('white_female', 'black_female', 'Emily Sullivan vs Lakisha Washington'),
]

test_results = []
for arm in arm_order:
    arm_valid = valid_df[valid_df['arm'] == arm]
    for ga, gb, label in race_pairs_test:
        pair_trials = arm_valid[
            ((arm_valid['first_group'] == ga) & (arm_valid['second_group'] == gb)) |
            ((arm_valid['first_group'] == gb) & (arm_valid['second_group'] == ga))
        ]
        if len(pair_trials) == 0:
            print(f'{ARM_LABELS[arm]:15s} | {label:45s} | {"--":>6s} | {"--":>6s} | {"0":>6s} | {"N/A":>10s} | --')
            continue
        white_wins = (pair_trials['selected_group'] == ga).sum()
        black_wins = (pair_trials['selected_group'] == gb).sum()
        total = white_wins + black_wins
        if total > 0:
            result = stats.binomtest(white_wins, total, 0.5)
            p = result.pvalue
            sig = '***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else ''
            print(f'{ARM_LABELS[arm]:15s} | {label:45s} | {white_wins:>6d} | {black_wins:>6d} | {total:>6d} | {p:>10.4f} | {sig}')
            test_results.append({'arm': arm, 'pair': label, 'white': white_wins,
                                'black': black_wins, 'total': total, 'p': p})

print('\n* p<0.05, ** p<0.01, *** p<0.001')

## 12. MCP Bias Reduction Summary

Quantify how much the MCP pipeline reduces racial disparity compared to the raw baseline.

In [ ]:
# Compute racial disparity for each arm
disparities = {}
for arm in arm_order:
    arm_valid = valid_df[valid_df['arm'] == arm]
    if len(arm_valid) == 0:
        disparities[arm] = {'disparity': None, 'white': None, 'black': None, 'n': 0}
        continue
    rates = compute_race_rates(arm_valid)
    w = rates.get('White', 0)
    b = rates.get('Black', 0)
    disparities[arm] = {'disparity': (w - b) * 100, 'white': w * 100, 'black': b * 100, 'n': len(arm_valid)}

fig, ax = plt.subplots(figsize=(10, 6))

arms_with_data = [a for a in arm_order if disparities[a]['disparity'] is not None]
disp_vals = [disparities[a]['disparity'] for a in arms_with_data]
colors = [ARM_COLORS[a] for a in arms_with_data]

bars = ax.bar([ARM_LABELS[a] for a in arms_with_data], disp_vals, color=colors, width=0.5)
for bar, val, arm in zip(bars, disp_vals, arms_with_data):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5 if val >= 0 else bar.get_height() - 2,
            f'{val:+.1f}pp', ha='center', va='bottom' if val >= 0 else 'top',
            fontsize=13, fontweight='bold')

ax.axhline(y=0, color='black', linewidth=1)
ax.set_ylabel('Racial Disparity\n(White rate - Black rate, percentage points)')
ax.set_title('Racial Disparity by Arm\n(0 = no disparity, positive = pro-White bias)',
             fontsize=14, fontweight='bold')
ax.set_ylim(min(min(disp_vals) - 10, -10), max(max(disp_vals) + 10, 10))

plt.tight_layout()
plt.savefig('results/fig11_disparity_reduction.png', bbox_inches='tight')
plt.show()

# Summary
print('\n=== BIAS REDUCTION SUMMARY ===')
for arm in arm_order:
    d = disparities[arm]
    if d['disparity'] is not None:
        print(f"\n{ARM_LABELS[arm]} (n={d['n']} valid trials):")
        print(f"  White selection rate: {d['white']:.1f}%")
        print(f"  Black selection rate: {d['black']:.1f}%")
        print(f"  Disparity: {d['disparity']:+.1f} percentage points")

if disparities['raw_naive']['disparity'] is not None and disparities['mcp']['disparity'] is not None:
    raw_disp = disparities['raw_naive']['disparity']
    mcp_disp = disparities['mcp']['disparity']
    reduction = raw_disp - mcp_disp
    reduction_pct = (reduction / abs(raw_disp)) * 100 if raw_disp != 0 else 0
    print(f"\n--- MCP vs Raw Naive ---")
    print(f"  Disparity reduction: {reduction:.1f} percentage points")
    print(f"  Relative reduction: {reduction_pct:.0f}%")

## 13. Refusal Rate Analysis

Refusal rates by arm and criteria tier — the model's safety training interacts with demographic visibility.

In [ ]:
# Refusal rate by arm and criteria
refusal_data = []
for arm in arm_order:
    arm_df_all = df[df['arm'] == arm]
    for tier in criteria_tiers:
        tier_df = arm_df_all[arm_df_all['criteria_tier'] == tier]
        total = len(tier_df)
        refused = (tier_df['selected_group'] == 'refused').sum()
        unparseable = (tier_df['selected_group'] == 'unparseable').sum()
        valid = total - refused - unparseable - (tier_df['selected_group'] == 'error').sum()
        refusal_data.append({
            'arm': arm, 'criteria': tier,
            'refusal_rate': refused / total * 100 if total > 0 else 0,
            'unparseable_rate': unparseable / total * 100 if total > 0 else 0,
            'valid_rate': valid / total * 100 if total > 0 else 0,
        })

ref_df = pd.DataFrame(refusal_data)

fig, axes = plt.subplots(1, 3, figsize=(16, 5), sharey=True)
tier_colors = {'female_stereotyped': '#e74c3c', 'male_stereotyped': '#3498db', 'neutral': '#95a5a6'}
tier_short = {'female_stereotyped': 'Nurturing', 'male_stereotyped': 'Tough', 'neutral': 'Neutral'}

for arm_idx, arm in enumerate(arm_order):
    arm_ref = ref_df[ref_df['arm'] == arm]
    x = np.arange(len(criteria_tiers))

    valid_vals = [arm_ref[arm_ref['criteria'] == t]['valid_rate'].values[0] for t in criteria_tiers]
    refused_vals = [arm_ref[arm_ref['criteria'] == t]['refusal_rate'].values[0] for t in criteria_tiers]
    unparse_vals = [arm_ref[arm_ref['criteria'] == t]['unparseable_rate'].values[0] for t in criteria_tiers]

    width = 0.6
    axes[arm_idx].bar(x, valid_vals, width, label='Valid', color='#2ecc71')
    axes[arm_idx].bar(x, refused_vals, width, bottom=valid_vals, label='Refused', color='#e74c3c')
    bottoms = [v + r for v, r in zip(valid_vals, refused_vals)]
    axes[arm_idx].bar(x, unparse_vals, width, bottom=bottoms, label='Unparseable', color='#95a5a6')

    axes[arm_idx].set_title(f'{ARM_LABELS[arm]}')
    axes[arm_idx].set_xticks(x)
    axes[arm_idx].set_xticklabels([tier_short[t] for t in criteria_tiers])
    axes[arm_idx].set_ylim(0, 110)
    if arm_idx == 0:
        axes[arm_idx].set_ylabel('Percentage of Trials')
    if arm_idx == 2:
        axes[arm_idx].legend(fontsize=9)

fig.suptitle('Response Outcome by Arm and Criteria Tier', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('results/fig12_refusal_by_criteria.png', bbox_inches='tight')
plt.show()

## 14. Summary Table

All key metrics in one table for easy reference.

In [ ]:
summary_rows = []
for arm in arm_order:
    arm_all = df[df['arm'] == arm]
    arm_valid = valid_df[valid_df['arm'] == arm]
    total = len(arm_all)
    n_valid = len(arm_valid)
    n_refused = (arm_all['selected_group'] == 'refused').sum()
    n_unparse = (arm_all['selected_group'] == 'unparseable').sum()

    race_rates = compute_race_rates(arm_valid) if n_valid > 0 else {}
    gender_rates = compute_gender_rates(arm_valid) if n_valid > 0 else {}

    first = (arm_valid['selected_group'] == arm_valid['first_group']).sum() if n_valid > 0 else 0

    rates = compute_selection_rates(arm_valid) if n_valid > 0 else {}
    max_rate = max(rates.values()) if rates else 0
    adverse_count = sum(1 for g in rates if rates[g] / max_rate < 0.80) if max_rate > 0 else 0

    summary_rows.append({
        'Arm': ARM_LABELS[arm],
        'Total Trials': total,
        'Valid': n_valid,
        'Refused': n_refused,
        'Unparseable': n_unparse,
        'Valid %': f'{n_valid/total*100:.0f}%' if total > 0 else 'N/A',
        'White Rate': f'{race_rates.get("White", 0)*100:.1f}%' if n_valid > 0 else 'N/A',
        'Black Rate': f'{race_rates.get("Black", 0)*100:.1f}%' if n_valid > 0 else 'N/A',
        'Race Gap': f'{(race_rates.get("White", 0) - race_rates.get("Black", 0))*100:+.1f}pp' if n_valid > 0 else 'N/A',
        'Male Rate': f'{gender_rates.get("Male", 0)*100:.1f}%' if n_valid > 0 else 'N/A',
        'Female Rate': f'{gender_rates.get("Female", 0)*100:.1f}%' if n_valid > 0 else 'N/A',
        '1st-Position %': f'{first/n_valid*100:.1f}%' if n_valid > 0 else 'N/A',
        'Adverse Impact Groups': adverse_count,
    })

summary_df = pd.DataFrame(summary_rows)
summary_df.set_index('Arm', inplace=True)

# Style the table
display(summary_df.T)
print('\n(Race Gap = White selection rate - Black selection rate; 0 = parity)')